# Experimental pipeline for GEQIE

## Setup

### Imports

In [2]:
import json
import subprocess
import tempfile

import pandas as pd

from concurrent import futures
from pathlib import Path
from typing import Any, Dict

In [3]:
#%load_ext ipyform
#%form_config --auto-detect 1

### Configurations

In [4]:
# @markdown #### Input and output paths
input_path = "../assets/test_images/grayscale" # @param {type:"string"}
output_path = "../.outputs"        # @param {type:"string"}


# @markdown #### Encoding 
encoding_method = "frqi"           # @param {type:"string"}
grayscale = True                   # @param {type:"boolean"}
n_shots = 1024                     # @param {type:"integer"}
return_padded_counts = True        # @param {type:"boolean"}

# @markdown #### Concurrency config
max_concurrency = None             # @param {type:"integer"}
# @markdown **NOTE:** Type `None` for default value of `max_concurrency` (number of all available system CPUs)

input_path = Path(input_path)

### Processing functions

In [5]:
# @title Processing Functions

def process_file(
        input_path: str, 
        encoding_method: str, 
        n_shots: int, 
        grayscale: bool,
        return_padded_counts: bool,
        workdir: str,
) -> Dict[str, Any]:
    process_result = subprocess.run(
        f"geqie simulate \
            --encoding {encoding_method} \
            --image {input_path} \
            --n-shots {n_shots} \
            --return-padded-counts {return_padded_counts} \
            --grayscale {grayscale}",
        capture_output=True,
    )

    if process_result.returncode:
        raise RuntimeError(process_result.stderr.decode("utf-8"))

    with tempfile.TemporaryFile(dir=workdir, mode="w", delete=False) as output_file:
        df = pd.DataFrame(
            columns=["file_name", "result"],
            data=[[
                input_path,
                json.dumps(json.loads(process_result.stdout.decode("utf-8")))
            ]]
        )
        df.to_csv(output_file, sep="\t", index=False)

    return df

def process_directory(
        input_path: Path, 
        encoding_method: str, 
        n_shots: int, 
        grayscale: bool,
        return_padded_counts: bool, 
        workdir: str,
) -> Dict[str, Any]:
    workdir_path = Path(workdir)
    input_files = input_path.glob("*")

    with futures.ThreadPoolExecutor() as executor:
        execute_futures = [
            executor.submit(
                process_file,
                input_file_path,
                encoding_method,
                n_shots,
                grayscale,
                return_padded_counts,
                workdir,
            ) for input_file_path in input_files
        ]

    for f in execute_futures:
        f.result()

    dfs = []
    for temp_file_path in workdir_path.glob("*"):
        dfs.append(pd.read_csv(temp_file_path, sep="\t"))
    
    return pd.concat(dfs)


### Run

In [6]:
# @title Run

def run():
    with tempfile.TemporaryDirectory() as workdir:
        if input_path.is_file():
            df = process_file(
                input_path=input_path,
                encoding_method=encoding_method,
                n_shots=n_shots,
                return_padded_counts=return_padded_counts,
                grayscale=grayscale,
                workdir=workdir,
            )
        if input_path.is_dir():
            df = process_directory(
                input_path=input_path,
                encoding_method=encoding_method,
                n_shots=n_shots,
                return_padded_counts=return_padded_counts,
                grayscale=grayscale,
                workdir=workdir,
            )

    print(df)

run()

FileNotFoundError: [Errno 2] No such file or directory: 'geqie simulate             --encoding frqi             --image ../assets/test_images/grayscale/test_image_1010.png             --n-shots 1024             --return-padded-counts True             --grayscale True'

In [12]:
! pip install -e ../

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [8]:
! geqie simulate             --encoding frqi             --image ../assets/test_images/grayscale/test_image_1010.png             --n-shots 1024             --return-padded-counts True             --grayscale True

{"000": 0, "001": 269, "010": 79, "011": 171, "100": 160, "101": 95, "110": 239, "111": 11}
